In [1]:
%run downoald_and_cleaning_data.ipynb

Downloading...
From (original): https://drive.google.com/uc?id=1UR_MC8Q5K4JSRnSUX5MQ-sIGPjLLnm9f
From (redirected): https://drive.google.com/uc?id=1UR_MC8Q5K4JSRnSUX5MQ-sIGPjLLnm9f&confirm=t&uuid=b680d6d6-68b4-4f8a-b4be-22a4b31fc76c
To: /home/onyxia/work/PSSD/Data/data.csv
100%|██████████| 363M/363M [00:06<00:00, 53.0MB/s] 


In [2]:
df = pd.read_csv("data_clean.csv")


In [3]:
df.dropna(subset=['texte'], inplace=True)

In [4]:
df['texte'] = df['texte'].str.lower()

In [5]:
import re
from tqdm import tqdm

tqdm.pandas()
df['texte_clean'] = df['texte'].str.lower().progress_apply(lambda x: re.sub(r'[^\w\s]', '', x))
df['titre_clean'] = df['titre'].str.lower().progress_apply(lambda x: re.sub(r'[^\w\s]', '', x))


100%|██████████| 11970/11970 [00:00<00:00, 154817.18it/s]


In [9]:
from transformers import pipeline
from datasets import Dataset

classifier = pipeline(
    "zero-shot-classification",
    model="mtheo/camembert-base-xnli",
    device=0,
    hypothesis_template="Cet article parle de {}."
)
candidate_labels = [
    "faits divers",
    "économie",
    "politique",
    "cinéma",
    "mode",
    "musique",
    "sport",
    "tribune"
]
def batch_classify(df, text_column='texte', batch_size=96):
    texts = df[text_column].fillna(" ").tolist()
    texts = [t if isinstance(t, str) and t.strip() != "" else " " for t in texts]
    dataset = Dataset.from_dict({"text": texts})
    
   
    def batching(batch):
        results = classifier(
            batch["text"],
            candidate_labels=candidate_labels,
            batch_size=batch_size
        )
        batch_themes = [res["labels"][0] for res in results]
        return {"theme": batch_themes}
    
  
    dataset = dataset.map(
        batching,
        batched=True,
        batch_size=batch_size
    )
    
    return dataset.to_pandas()

Device set to use cuda:0


In [12]:
def filter_empty_lines(df, text_column='texte_clean'):
    filtered_df = df[~(df[text_column].isna() | (df[text_column].str.strip() == ''))].copy()
    filtered_df = filtered_df.reset_index()  
    print(f"Lignes initiales : {len(df)}")
    print(f"Lignes après filtrage : {len(filtered_df)}")
    return filtered_df

df_filtered = filter_empty_lines(df, text_column='texte_clean')


Lignes initiales : 11970
Lignes après filtrage : 11970


In [14]:
results = batch_classify(df_filtered, text_column='texte_clean', batch_size=164)

Map: 100%|██████████| 11970/11970 [1:00:05<00:00,  3.32 examples/s]


In [15]:
df_filtered['theme'] = results['theme'].values

In [16]:
df_filtered.tail(10)

,index,Unnamed: 0,identifiant,journal_clean,titre,annee,mois,jour,texte,keywords,texte_clean,titre_clean,theme
11960,11969,82068,533587a067dfeb0b1b2b14e238ff6320f28845770f2ed0...,Le Figaro,Harvey Weinstein conduit en urgence à l’hôpita...,2024,12,2,le producteur roi déchu d’hollywood harvey wei...,"weinstein, producteur, sanguins, new, tests, h...",le producteur roi déchu dhollywood harvey wein...,harvey weinstein conduit en urgence à lhôpital...,faits divers
11961,11970,82126,a3c235da28477a6045b1e97abd2b5491b1aa772c273ae4...,Libération,Me Too : Harvey Weinstein hospitalisé après de...,2024,12,3,le producteur roi déchu de hollywood harvey we...,"weinstein, producteur, fin, prison, harvey, se...",le producteur roi déchu de hollywood harvey we...,me too harvey weinstein hospitalisé après des...,faits divers
11962,11971,82129,20ae3e5ca278db0d33448092b67a7bb6e896e3e70436d8...,Le Monde,"Face aux violences sexuelles, l’examen de cons...",2024,12,3,leur parole était attendue. le festival de can...,"violences, cinéma, conscience, l’assemblée, in...",leur parole était attendue le festival de cann...,face aux violences sexuelles lexamen de consci...,cinéma
11963,11972,82137,377988b4b18f66bdf316259434127bc49e39ac016edacd...,Le Figaro,Italie: un homme condamné à la prison à vie po...,2024,12,3,la cour d'assises de venise a condamné mardi 3...,"relancé, pays, venise, cour, débat, féminicide",la cour dassises de venise a condamné mardi 3 ...,italie un homme condamné à la prison à vie pou...,faits divers
11964,11973,82155,1a7689437dc0d5a11e37df86ddd7d9ea4c37195a7ecdbc...,Le Point,Harvey Weinstein est de nouveau hospitalisé,2024,12,4,harvey weinstein est de retour à l'hôpital. l'...,"producteur, new, harvey, york, hospitalisé, hô...",harvey weinstein est de retour à lhôpital léqu...,harvey weinstein est de nouveau hospitalisé,cinéma
11965,11974,82156,e36f02ac7daaf39e2d3b66037ec45f1884593b1e6b73dd...,Libération,Devenir acteur,2024,12,4,"et si, en chemin, l'éducation populaire avait ...","acteur, populaire, éducation",et si en chemin léducation populaire avait un ...,devenir acteur,politique
11966,11975,82157,e021e6c61999c0fb494e2a49ed9614758d1e8318230a25...,Le Point,Quand l'air de Bohème souffle sur le Rocher,2024,12,4,"à 88 ans, charles dutoit a ralenti son activit...","air, dutoit, charles",à 88 ans charles dutoit a ralenti son activité...,quand lair de bohème souffle sur le rocher,musique
11967,11976,82172,ebd597bcedd9a52c354cd8337257ec2511ff3ae3b56296...,Libération,"A Marseille, devenir acteur de l’éducation pop...",2024,12,4,"et si, en chemin, l’éducation populaire avait ...","acteur, populaire, l’éducation",et si en chemin léducation populaire avait un ...,a marseille devenir acteur de léducation popul...,sport
11968,11977,82173,20d7c133c259fe5e149d1cddfa8be63aa616233362596f...,Le Figaro,«Adèle Haenel et ses 12 ans étaient d’une sens...,2024,12,4,au dernier étage d’un immeuble de la rue jean-...,"violences, christophe, adèle, réalisateur, d’u...",au dernier étage dun immeuble de la rue jeanpi...,adèle haenel et ses 12 ans étaient dune sensua...,faits divers
11969,11978,82175,4551bb033bd0882777b6f20a6c74d7cea3db52bdf498c5...,Libération,«Se libérer de Barbe-Bleue» d’Edith Lombardi :...,2024,12,4,"il était une fois, en ecosse, une pauvre veuve...","lombardi, barbe-bleue, fois, sœur, vient, s’en...",il était une fois en ecosse une pauvre veuve q...,se libérer de barbebleue dedith lombardi sœur...,faits divers


In [17]:
df_faits = df_filtered[df_filtered['theme']=='faits divers']

In [18]:
df_faits.shape

(5676, 13)

In [ ]:
import pandas as pd
df_faits=pd.read_csv('data_faits_divers.csv')

In [ ]:
import matplotlib.pyplot as plt
articles_par_annee = df_faits['annee'].value_counts().sort_index()
plt.figure(figsize=(10, 6))
articles_par_annee.plot(kind='bar')
plt.title("Répartition des articles par année")
plt.xlabel("Année")
plt.ylabel("Nombre d'articles")
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()
plt.show()

In [ ]:
articles_par_annee = df_faits['annee'].value_counts(normalize=True).sort_index() * 100
plt.figure(figsize=(10, 6))
articles_par_annee.plot(kind='bar')
plt.title("Répartition des articles par année (%)")
plt.xlabel("Année")
plt.ylabel("Pourcentage d'articles")
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()
plt.show()
plt.savefig('plot.png')

In [19]:
df_faits.to_csv('data_faits_divers.csv')

In [20]:
df_filtered.to_csv('data_all.csv')